Homework 2: n-gram LM
----

Due date: 25 Feb 2024

Points: 105

Goals:
- understand the difficulties of counting and probablities in NLP applications
- work with real world data to build a functioning language model
- stress test your model (to some extent)

Complete in groups of: __one (individually)__

Allowed python modules:
- `numpy`, `matplotlib`, and all built-in python libraries (e.g. `math` and `string`)
- do not use `nltk` or `pandas`

Instructions:
- Complete outlined problems in this notebook. 
- When you have finished, __clear the kernel__ and __run__ your notebook "fresh" from top to bottom. Ensure that there are __no errors__. 
    - If a problem asks for you to write code that does result in an error (as in, the answer to the problem is an error), leave the code in your notebook but commented out so that running from top to bottom does not result in any errors.
- Double check that you have completed Task 0.
- Submit your work on Gradescope.
- Double check that your submission on Gradescope looks like you believe it should.

Task 0: Name, References, Reflection (5 points)
---

Name: Nisharg Gosai

References
---
List the resources you consulted to complete this homework here. Write one sentence per resource about what it provided to you. If you consulted no references to complete your assignment, write a brief sentence stating that this is the case and why it was the case for you.

(Example)
- https://docs.python.org/3/tutorial/datastructures.html
    - Read about the the basics and syntax for data structures in python.
- https://www.youtube.com/watch?v=GiyMGBuu45w
    - Understanding n gram models
- https://www.w3schools.com/python/python_file_handling.asp
    - File handling in python for reference

AI Collaboration
---
Following the *Policy on the use of Generative AI* in the syllabus, please cite any LLMs that you used here and briefly describe what you used them for. Additionally, provide comments in-line identifying the specific sections that you used LLMs on, if you used them towards the generation of any of your answers.

OpenAI. (2023). ChatGPT 3.5 [Large language model]. https://chat.openai.com/chat
- used as a quick reference handbook for things like python documentation and mathematical formulas

Reflection
----
Answer the following questions __after__ you complete this assignment (no more than 1 sentence per question required, this section is graded on completion):

1. Does this work reflect your best effort?
   Yes
2. What was/were the most challenging part(s) of the assignment?
   Debugging and conversions 
3. If you want feedback, what function(s) or problem(s) would you like feedback on and why?
   My training data from 1st hw part

Task 1: Berp Data Write-Up (5 points)
---

Every time you use a data set in an NLP application (or in any software application), you should be able to answer a set of questions about that data. Answer these now. Default to no more than 1 sentence per question needed. If more explanation is necessary, do give it.

This is about the __berp__ data set.

1. Where did you get the data from? https://www1.icsi.berkeley.edu/Speech/berp.html 
    - the dataset can be found at https://github.com/wooters/berp-trans 
2. How was the data collected (where did the people acquiring the data get it from and how)? 
    - speech was recorded from test subjects at icsi. The audio data was recorded with a Sennheiser close-talking microphone and was sampled at 16 kHz.
3. How large is the dataset? (# lines, # tokens)
    - 8566 utterances, with about 1900 unique words
4. What is your data? (i.e. newswire, tweets, books, blogs, etc)
    - speech recorded from test subjects, about 7 hours of speech
5. Who produced the data? (who were the authors of the text? Your answer might be a specific person or a particular group of people)
    - Daniel Jurafsky, Chuck Wooters, Gary Tajchman, Jonathan Segal, Andreas Stolcke, Eric Fosler, and Nelson Morgan

Task 2: Implement an n-gram Language Model (90 points)
----

Implement the `LanguageModel` class as outlined in the provided `lm_model.py` file. Do not change function signatures (the unit tests that we provide and in the autograder will break).

Your language model:
- *must* work for the unigram, bigram, and trigram cases (5 points are allocated to an experiment involving larger values of `n`)
    - hint: try to implement the bigram case as a generalized "n greater than 1" case
- should be *token agnostic* (this means that if we give the model text tokenized as single characters, it will function as a character language model and if we give the model text tokenized as "words" (or "traditionally"), then it will function as a language model with those tokens)
- will use Laplace smoothing
- will replace all tokens that occur only once with `<UNK>` at train time
    - do not add `<UNK>` to your vocabulary if no tokens in the training data occur only once!

We have provided:
- a function to read in files
- some functions to change a list of strings into tokens
- the skeleton of the `LanguageModel` class

You need to implement:
- all functions marked

You may implement:
- additional functions/methods as helpful to you

As a general guideline for reference, including comments, all code required, and some debugging code that can be run with `verbose` parameters, our solution is ~ 300 lines. (~ 120 lines added to the starter code).

Points breakdown marked in code below.

In [1]:
import lm_model as lm
import numpy as np
import matplotlib.pyplot as plt
import statistics

In [2]:
# test the language model (unit tests)
import test_minitrainingprovided as test

# passing all these tests is a good indication that your model
# is correct. They are *not a guarantee*, so make sure to look
# at the tests and the cases that they cover. (we'll be testing
# your model against all of the testing data in addition).

# autograder points in gradescope are assigned SIXTY points
# this is essentially 60 points for correctly implementing your
# underlying model
# there are an additional 10 points manually graded for the correctness
# parts of your sentence generation

# make sure all training files are in a "training_files" directory 
# that is in the same directory as this notebook

unittest = test.TestMiniTraining()
unittest.test_createunigrammodellaplace()
unittest.test_createbigrammodellaplace()
unittest.test_unigramlaplace()
unittest.test_unigramunknownslaplace()
unittest.test_bigramlaplace()
unittest.test_bigramunknownslaplace()
# produces output
unittest.test_generateunigramconcludes()
# produces output
unittest.test_generatebigramconcludes()

unittest.test_onlyunknownsgenerationandscoring()

[['<s>', 'i', 'am', '</s>'], ['<s>', 'sam', '</s>']]
[['<s>', 'i', 'am', 'ham', 'i', 'am', 'ham', 'i', 'am', '</s>'], ['<s>', 'ham', 'i', 'am', '</s>']]


In [3]:
# 5 points

# instantiate a bigram language model, train it, and generate ten sentences

# make sure your output is nicely formatted!

ngram = 2
training_file_path = "training_files/berp-training.txt"

# optional parameter tells the tokenize function how to tokenize
by_char = False
data = lm.read_file(training_file_path)
tokens = lm.tokenize(data, ngram, by_char=False)

# # YOUR CODE HERE
bigram_model = lm.LanguageModel(ngram)

# Train the language model
bigram_model.train(tokens)

# Generate ten sentences
num_sentences = 10
generated_sentences = bigram_model.generate(num_sentences)

print("Generated Sentences:")
for i, sentence in enumerate(generated_sentences, 1):
    print(f"{i}. {' '.join(sentence)}")


Generated Sentences:
1. <s> i want to be at most a reservation for lunch in having more information about sushi </s>
2. <s> <UNK> </s>
3. <s> do you have a restaurant </s>
4. <s> uh which restaurant with a list all </s>
5. <s> i would like to eat on wednesday i don't want to know of this friday night or fifteen minutes' driving distance to spend </s>
6. <s> i will be vegetarian restaurants within two hundred meters meters meters </s>
7. <s> sunday </s>
8. <s> i could be no more </s>
9. <s> start again please </s>
10. <s> i would like german food is okay i want to travel for a car so my name of the distance from icksee please </s>


In [4]:
# 5 points

# evaluate your bigram model on the test data
# score each line in the test data individually, then calculate the average score
# you need not re-train your model
test_path = "testing_files/berp-test.txt"
test_data = lm.read_file(test_path)

scores = []

# YOUR CODE HERE
scores = [bigram_model.score(lm.tokenize_line(sentence, ngram, by_char)) for sentence in test_data]

# Print out the mean score and standard deviation
# for words-as-tokens, these values should be
# ~4.9 * 10^-5 and 0.000285
print("Average = ", sum(scores)/len(scores))
print("Standard Deviation = ",statistics.stdev(scores))

Average =  4.962082362726267e-05
Standard Deviation =  0.000286735365135695


In [5]:
import timeit
start_time = timeit.default_timer()
# 5 points
# see if you can train your model on the data you found for your first homework
# what is the maximum value of n <= 10 that you can train a model *in your programming environment* in a reasonable amount of time? (less than 3 - 5 minutes)
# generate three sentences with this model

import json
ngram = 7
training_file_path = "training_files/Sarcasm_Headlines_Dataset_v2.json"
num_sentences_to_generate = 5  

# Read JSON data line by line
json_data = []
with open(training_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        entry = json.loads(line)
        json_data.append(entry)

# Extract headlines from JSON data
headlines = [entry['headline'] for entry in json_data]

# Train the final language model
final_model = lm.LanguageModel(ngram)
final_model.train(lm.tokenize(headlines, ngram, by_char=False))  

# Generate sentences with the final model
generated_sentences = final_model.generate(num_sentences_to_generate)

# Print generated sentences
print("\nGenerated Sentences:")
for i, sentence in enumerate(generated_sentences, 1):
    print(f"{i}. {' '.join(sentence)}")

elapsed = timeit.default_timer() - start_time
print(elapsed)



Generated Sentences:
1. <s> <s> <s> <s> <s> <s> quick, <UNK> death tops holiday wish list of local veal calf </s> </s> </s> </s> </s> </s>
2. <s> <s> <s> <s> <s> <s> gop <UNK> fantasy is alive and well </s> </s> </s> </s> </s> </s>
3. <s> <s> <s> <s> <s> <s> <UNK> davis makes powerful demand on behalf of women of color at women's march </s> </s> </s> </s> </s> </s>
4. <s> <s> <s> <s> <s> <s> rock & roll hall of fame rescinds nomination after discovering the cure was voted in as cruel prank by popular kids </s> </s> </s> </s> </s> </s>
5. <s> <s> <s> <s> <s> <s> donald trump calls kim jong un a 'smart cookie' </s> </s> </s> </s> </s> </s>
0.5305656999989878


Implement the corresponding function and evaluate the perplexity of your model on the first 20 lines in the test data for values of `n` from 1 to 3. Perplexity should be individually calculated for each line.

In [6]:
test_path = "testing_files/berp-test.txt"
test_data = lm.read_file(test_path)

for ngram in range(1, 4):
    print("********")
    print("Ngram model:", ngram)

    # Tokenize test data
    test_tokens = lm.tokenize(test_data, ngram, by_char=False)

    # Instantiate and train the language model
    ngram_model = lm.LanguageModel(ngram)
    ngram_model.train(lm.tokenize(test_tokens, ngram, by_char=False))

    # Evaluate perplexity for each line in the test data
    perplexities = []
    for line_tokens in test_tokens[:20]:
        perplexity = ngram_model.perplexity(line_tokens)
        perplexities.append(perplexity)

    # Calculate and print mean perplexity
    mean_perplexity = sum(perplexities) / len(perplexities)
    print(f"Ngram: {ngram} Mean Perplexity: {mean_perplexity:.4f}")



********
Ngram model: 1
Ngram: 1 Mean Perplexity: 36.5782
********
Ngram model: 2
Ngram: 2 Mean Perplexity: 49.7073
********
Ngram model: 3
Ngram: 3 Mean Perplexity: 10.0350


1. What are the common attributes of the test sentences that cause very high perplexity? long sentences, unusual sentence structure, rare words

5 points in this assignment are reserved for overall style (both for writing and for code submitted). All work submitted should be clear, easily interpretable, and checked for spelling, etc. (Re-read what you write and make sure it makes sense). Course staff are always happy to give grammatical help (but we won't pre-grade the content of your answers).